# EDA for cases data from 2015 to 2021

In [143]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns

## Basic exploration
We begin by loading the data and looking at its basic shape:

In [144]:
df = pd.read_csv('../data/processed/MVAallcases_cleaned.csv')

In [145]:
df.shape

(6602407, 25)

We display the basic cases data:

In [146]:
pd.options.display.max_columns = 100
df.head(5)

,Officer Dispatch UID,Dispatch ID,CAD Event ID,Officer Serial Num,AS Of Officer Title,As_On_Officer Precinct_ID,As_On_Officer Precinct_Desc,As_On_Officer Squad_Desc,Partner Officer Serial Num,Clear By Desc,Call Type Desc,Case Type Initial Desc,Case Type Final Desc,Call Priority Code,GO Num,First Dispatch Time,Clear Time,Total Service Time,Precinct,Sector,Beat,Dispatch Address,Blurred_Latitude,Blurred_Longitude,Year
0,2016000447501V27651,2016000447501V2,2016000447501,7651,POLICE OFFICER,B110,WEST PCT,WEST PCT 2ND W - MARY BEATS,7651,ORAL WARNING GIVEN,ONVIEW,PEDESTRIAN VIOLATIONS,--TRAFFIC - PEDESTRIAN VIOLATION,3,20160000447501,2016-12-13 16:30:25.00,2016-12-13 16:34:03.00,3.0,WEST,MARY,M2,3 AV / PINE ST,47.610743,-122.338702,2016
1,20160002116652R337744,20160002116652R33,2016000211665,7744,POLICE OFFICER,B230,SOUTH PCT,SOUTH PCT 2ND W - ROBERT,7744,PHYSICAL ARREST MADE,ONVIEW,"NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)",--NARCOTICS - NARS REPORT,2,20160000211665,2016-06-14 16:20:29.00,2016-06-14 21:03:43.00,41.0,SOUTH,ROBERT,R3,4316 S GENESEE ST,47.564089,-122.278254,2016
2,2016000211088T335251,2016000211088T33,2016000211088,5251,POLICE OFFICER,B630,TRAFFIC SECTION,TRAF - MOTORCYCLE UNIT - T3 SQUAD,-,ASSISTANCE RENDERED,911,"MVC - REPORT, NON INJ/NON BLKG OR AFTER FACT INJ",--TRAFFIC - MV COLLISION INVESTIGATION,3,20160000211088,2016-06-14 08:09:29.00,2016-06-14 12:28:56.00,56.0,NORTH,BOY,B2,8 AV NW / NW 56 ST,47.669428,-122.366179,2016
3,20160003864582U017497,20160003864582U01,2016000386458,7497,POLICE OFFICER,B120,NORTH PCT,NORTH PCT 2ND W - L/U RELIEF,7557,REPORT WRITTEN (NO ARREST),911,DIST - DV - NO ASLT,"--DV - ARGUMENTS, DISTURBANCE (NO ARREST)",2,20160000386458,2016-10-25 14:26:00.00,2016-10-25 15:01:16.00,35.0,NORTH,UNION,U3,5114 26 AV NE,47.666846,-122.299287,2016
4,20160004354893G27752,20160004354893G2,2016000435489,7752,POLICE OFFICER,B240,EAST PCT,EAST PCT 3RD W - GEORGE,7776,ASSISTANCE RENDERED,911,SERVICE - WELFARE CHECK,--DISTURBANCE - OTHER,3,20160000435489,2016-12-03 20:10:25.00,2016-12-03 22:38:46.00,15.0,EAST,GEORGE,G2,140 23 AV S[MONICA'S VILLAGE 403,47.601002,-122.302236,2016


### Columns description
1. ```Officer Dispatch UID```: Dispatch ID + Officer Serial Num
2. ```Dispatch ID```: Dispatch ID
3. ```CAD Event ID```: computer aided dispatch (CAD) event ID number is generated when the service is initiated
4. ```Officer Serial Num```: serial number of the officer 
5. ```AS Of Officer Title```: Officer title
6. ```As_On_Officer Precinct_ID```: precinct ID that dispatched the officer
7. ```As_On_Officer Precinct_Desc```: description of the precinct
8. ```As_On_Officer Squad_Desc```: ?
9. ```Partner Officer Serial Num```: serial number of the partner officer
10. ```Clear By Desc```: how was the case cleared
11. ```Call Type Desc```: type of call that initiated the case
12. ```Case Type Initial Desc```: initial description of the case
13. ```Case Type Final Desc```: final description of the case
14. ```Call Priority Code```: ?
15. ```GO Num```: ID assigned when report is written and submitted
16. ```First Dispatch Time```: time at dispatch
17. ```Clear Time```: time at event was cleared
18. ```Total Service Time```: in min
19. ```Precinct```: precint name (loc?)
20. ```Sector```: ?
21. ```Beat```: ?
22. ```Dispatch Address```: address from which officer was dispatched
23. ```Blurred_Latitude```: ?
24. ```Blurred_Longitude```: ?
25. ```Year```: year of the incident

### Remove NaN values

In [147]:
#Check on the % of null values
df.isnull().sum()*100/df.shape[0]

Officer Dispatch UID           0.000000
Dispatch ID                    0.000000
CAD Event ID                   0.000000
Officer Serial Num             0.000000
AS Of Officer Title            0.000000
As_On_Officer Precinct_ID      0.000000
As_On_Officer Precinct_Desc    0.000000
As_On_Officer Squad_Desc       0.000000
Partner Officer Serial Num     0.000000
Clear By Desc                  0.000000
Call Type Desc                 0.000000
Case Type Initial Desc         0.000000
Case Type Final Desc           0.000000
Call Priority Code             0.000000
GO Num                         0.000000
First Dispatch Time            0.000000
Clear Time                     0.000000
Total Service Time             0.047589
Precinct                       0.000000
Sector                         0.695125
Beat                           0.000000
Dispatch Address               0.000000
Blurred_Latitude               0.000000
Blurred_Longitude              0.000000
Year                           0.000000


In [148]:
# Drop NaN values
df = df.dropna()

In [149]:
df.isnull().sum()*100/df.shape[0]

Officer Dispatch UID           0.0
Dispatch ID                    0.0
CAD Event ID                   0.0
Officer Serial Num             0.0
AS Of Officer Title            0.0
As_On_Officer Precinct_ID      0.0
As_On_Officer Precinct_Desc    0.0
As_On_Officer Squad_Desc       0.0
Partner Officer Serial Num     0.0
Clear By Desc                  0.0
Call Type Desc                 0.0
Case Type Initial Desc         0.0
Case Type Final Desc           0.0
Call Priority Code             0.0
GO Num                         0.0
First Dispatch Time            0.0
Clear Time                     0.0
Total Service Time             0.0
Precinct                       0.0
Sector                         0.0
Beat                           0.0
Dispatch Address               0.0
Blurred_Latitude               0.0
Blurred_Longitude              0.0
Year                           0.0
dtype: float64

### Explore variables

Evaluate `CAD Event ID`

In [150]:
#len(df['CAD Event ID'].unique())
df['CAD Event ID'].astype('category').value_counts()

2020000000000    953543
2019000000000      3274
2016000083081       421
2017000023845       311
2016000468678       258
                  ...  
2017000395900         1
2016000339721         1
2016000339720         1
2016000339719         1
2018000252211         1
Name: CAD Event ID, Length: 1813077, dtype: int64

In [151]:
df[df['CAD Event ID']==2020000000000].head(5)

,Officer Dispatch UID,Dispatch ID,CAD Event ID,Officer Serial Num,AS Of Officer Title,As_On_Officer Precinct_ID,As_On_Officer Precinct_Desc,As_On_Officer Squad_Desc,Partner Officer Serial Num,Clear By Desc,Call Type Desc,Case Type Initial Desc,Case Type Final Desc,Call Priority Code,GO Num,First Dispatch Time,Clear Time,Total Service Time,Precinct,Sector,Beat,Dispatch Address,Blurred_Latitude,Blurred_Longitude,Year
3145498,20200002327603M18664,20200002327603M1,2020000000000,8664,POLICE OFFICER PROBATION,B110,WEST PCT,WEST PCT 3RD W - D/M RELIEF,8687,PHYSICAL ARREST MADE,911,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),"ASSAULTS, OTHER",1,20200000000000,25:40.0,10:55.0,54.0,WEST,DAVID,D3,1812 BOREN AV,47.616558,-122.332067,2020
3145499,20200001763902Q38611,20200001763902Q3,2020000000000,8611,POLICE OFFICER,B110,WEST PCT,WEST PCT 2ND W - QUEEN,8576,UNABLE TO LOCATE INCIDENT OR COMPLAINANT,911,"SUSPICIOUS PERSON, VEHICLE OR INCIDENT",SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,2,20200000000000,03:12.0,34:44.0,13.0,WEST,QUEEN,Q2,1401 BLOCK W BERTONA ST [ALLEY TO SOUTH,-1.000000,-1.000000,2020
3145500,20200001479413O118390,20200001479413O11,2020000000000,8390,POLICE OFFICER,B230,SOUTH PCT,SOUTH PCT 3RD W - OCEAN RELIEF,-,REPORT WRITTEN (NO ARREST),911,SFD - ASSIST ON FIRE OR MEDIC RESPONSE,ASSIST OTHER AGENCY - CITY AGENCY,1,20200000000000,29:35.0,15:42.0,39.0,SOUTH,OCEAN,O1,503 S ROYAL BROUGHAM WY,47.592397,-122.327325,2020
3145501,20200001962012O037709,20200001962012O03,2020000000000,7709,POLICE OFFICER,B230,SOUTH PCT,SOUTH PCT 2ND W - OCEAN RELIEF,8804,ASSISTANCE RENDERED,911,UNKNOWN - ANI/ALI - WRLS PHNS (INCL OPEN LINE),--ASSIST PUBLIC - OTHER (NON-SPECIFIED),1,20200000000000,30:57.0,05:59.0,1.0,SOUTH,OCEAN,O2,4000 AIRPORT WY S,47.567635,-122.322665,2020
3145502,20200000465052E28336,2.02E+15,2020000000000,8336,POLICE OFFICER,B240,EAST PCT,EAST PCT 2ND W - EDWARD,-,ASSISTANCE RENDERED,911,"DISTURBANCE, MISCELLANEOUS/OTHER",PROWLER - TRESPASS,2,20200000000000,51:23.0,05:31.0,22.0,EAST,EDWARD,E2,1125 12 AV [URBAN STORAGE,47.612226,-122.316756,2020


In [152]:
df[df['CAD Event ID']==2020000000000]['Case Type Initial Desc'].astype('category').value_counts()

DISTURBANCE, MISCELLANEOUS/OTHER                62620
SUSPICIOUS PERSON, VEHICLE OR INCIDENT          53144
TRESPASS                                        52067
PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY    46936
DIRECTED PATROL ACTIVITY                        38214
                                                ...  
JUVENILE - HARBORING A MINOR                        4
ESCAPE - PRISONER                                   3
TRU - PURSE SNATCH                                  2
SECONDARY - FORGERY/BUNCO/SCAMS/ID THEFT            1
SECONDARY - THEFT (NOT SHOPLIFT OR SERVICES)        1
Name: Case Type Initial Desc, Length: 232, dtype: int64

In [153]:
df[df['CAD Event ID']==2016000083081].head(5)

,Officer Dispatch UID,Dispatch ID,CAD Event ID,Officer Serial Num,AS Of Officer Title,As_On_Officer Precinct_ID,As_On_Officer Precinct_Desc,As_On_Officer Squad_Desc,Partner Officer Serial Num,Clear By Desc,Call Type Desc,Case Type Initial Desc,Case Type Final Desc,Call Priority Code,GO Num,First Dispatch Time,Clear Time,Total Service Time,Precinct,Sector,Beat,Dispatch Address,Blurred_Latitude,Blurred_Longitude,Year
9390,20160000830811E37675,20160000830811E3,2016000083081,7675,POLICE OFFICER,B240,EAST PCT,EAST PCT 3RD W - EDWARD,7675,REPORT WRITTEN (NO ARREST),ONVIEW,EXPLOSION - IP/JO,--ASSIST OTHER AGENCY - CITY AGENCY,1,20160000083081,2016-03-09 01:43:31.00,2016-03-16 15:22:29.00,55.0,NORTH,JOHN,J2,GREENWOOD AV N / N 85 ST,47.690594,-122.355302,2016
14784,20160000830812Y657799,20160000830812Y65,2016000083081,7799,POLICE OFFICER PROBATION,B230,SOUTH PCT,SOUTH PCT 2ND W - ROBERT,7799,REPORT WRITTEN (NO ARREST),ONVIEW,EXPLOSION - IP/JO,--ASSIST OTHER AGENCY - CITY AGENCY,1,20160000083081,2016-03-09 01:43:31.00,2016-03-16 15:22:29.00,7.0,NORTH,JOHN,J2,GREENWOOD AV N / N 85 ST,47.690594,-122.355302,2016
17069,20160000830813Y618323,20160000830813Y61,2016000083081,8323,POLICE OFFICER PROBATION,B120,NORTH PCT,NORTH PCT 2ND W - BOY,4697,REPORT WRITTEN (NO ARREST),ONVIEW,EXPLOSION - IP/JO,--ASSIST OTHER AGENCY - CITY AGENCY,1,20160000083081,2016-03-09 01:43:31.00,2016-03-16 15:22:29.00,39.0,NORTH,JOHN,J2,GREENWOOD AV N / N 85 ST,47.690594,-122.355302,2016
21799,20160000830813Y635467,20160000830813Y63,2016000083081,5467,POLICE OFFICER,C310,COMMUNICATIONS SECTION,COMM - INTERNET AND TELEPHONE REPORTING (ITRU),5467,REPORT WRITTEN (NO ARREST),ONVIEW,EXPLOSION - IP/JO,--ASSIST OTHER AGENCY - CITY AGENCY,1,20160000083081,2016-03-09 01:43:31.00,2016-03-16 15:22:29.00,43.0,NORTH,JOHN,J2,GREENWOOD AV N / N 85 ST,47.690594,-122.355302,2016
24555,20160000830812Y614583,20160000830812Y61,2016000083081,4583,POLICE SERGEANT,B120,NORTH PCT,NORTH PCT 1ST W - NORA (JOHN),4047,REPORT WRITTEN (NO ARREST),ONVIEW,EXPLOSION - IP/JO,--ASSIST OTHER AGENCY - CITY AGENCY,1,20160000083081,2016-03-09 01:43:31.00,2016-03-16 15:22:29.00,32.0,NORTH,JOHN,J2,GREENWOOD AV N / N 85 ST,47.690594,-122.355302,2016


In [154]:
df[df['CAD Event ID']==2016000083081]['Case Type Initial Desc'].astype('category').value_counts()

EXPLOSION - IP/JO    421
Name: Case Type Initial Desc, dtype: int64

In [155]:
# Drop rows for CAD ID = 2020000000000, 2019000000000
df.drop(df.loc[df['CAD Event ID'].isin([2020000000000,2019000000000])].index, inplace=True)

In [156]:
df['CAD Event ID'].astype('category').value_counts()

2016000083081    421
2017000023845    311
2016000468678    258
2017000018921    212
2015000227233    205
                ... 
2016000339723      1
2017000395900      1
2016000339721      1
2016000339720      1
2018000380801      1
Name: CAD Event ID, Length: 1813075, dtype: int64

In [157]:
df['Case Type Initial Desc'].astype('category').value_counts()
df['Case Type Final Desc'].astype('category').value_counts()

--DISTURBANCE - OTHER                       559251
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON    459392
--PREMISE CHECKS - CRIME PREVENTION         420320
--ASSIST PUBLIC - OTHER (NON-SPECIFIED)     256256
--TRAFFIC - MOVING VIOLATION                229928
                                             ...  
AWOL - ADULT OR JUVENILE                         1
DV -VULNERABLE ADULT, PHYSICAL ABUSE             1
KNOWN KIDNAPPNG                                  1
TRU - ILLEGAL DUMPING                            1
SECONDARY - FORGERY/BUNCO/SCAMS/ID THEFT         1
Name: Case Type Final Desc, Length: 511, dtype: int64

In [158]:
var_df = df.drop_duplicates(subset=['CAD Event ID', 'Case Type Initial Desc'], keep='first')
var = var_df.groupby(['Case Type Initial Desc', 'Year']).size().reset_index()
var

,Case Type Initial Desc,Year,0
0,-,2021,370
1,"--ASSAULTS, OTHER",2017,2
2,--MISCHIEF OR NUISANCE - GENERAL,2018,1
3,--NARCOTICS - OTHER,2016,1
4,--ROBBERY - STRONG ARM,2016,1
...,...,...,...
1458,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2016,431
1459,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2017,516
1460,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2018,529
1461,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2019,699


In [159]:
var_df = df.drop_duplicates(subset=['CAD Event ID'], keep='first')
var = var_df.groupby(['Case Type Initial Desc', 'Year']).size().reset_index()
var

,Case Type Initial Desc,Year,0
0,-,2021,370
1,"--ASSAULTS, OTHER",2017,2
2,--MISCHIEF OR NUISANCE - GENERAL,2018,1
3,--NARCOTICS - OTHER,2016,1
4,--ROBBERY - STRONG ARM,2016,1
...,...,...,...
1458,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2016,431
1459,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2017,516
1460,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2018,529
1461,"WIRES DOWN (PHONE, ELECTRICAL,ETC.)",2019,699


In [160]:
#g = sns.FacetGrid(var, col='Year', col_wrap=2, height=5, aspect=1.5, sharex=False, sharey=False)
#g.map(sns.barplot, "Case Type Initial Desc", 0)

In [161]:
len(df['CAD Event ID'].unique())
len(df['Case Type Initial Desc'].unique())
len(df['Case Type Final Desc'].unique())

511

In [162]:
bor_hour.head()

,Case Type Initial Desc,Year,0
0,"--ASSAULTS, OTHER",2017,5
1,--MISCHIEF OR NUISANCE - GENERAL,2018,4
2,--NARCOTICS - OTHER,2016,1
3,--ROBBERY - STRONG ARM,2016,4
4,--TRAFFIC - MV COLLISION INVESTIGATION,2019,2


In [165]:
df['Case Type Final Desc'].astype('category').value_counts()

--DISTURBANCE - OTHER                       559251
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON    459392
--PREMISE CHECKS - CRIME PREVENTION         420320
--ASSIST PUBLIC - OTHER (NON-SPECIFIED)     256256
--TRAFFIC - MOVING VIOLATION                229928
                                             ...  
AWOL - ADULT OR JUVENILE                         1
DV -VULNERABLE ADULT, PHYSICAL ABUSE             1
KNOWN KIDNAPPNG                                  1
TRU - ILLEGAL DUMPING                            1
SECONDARY - FORGERY/BUNCO/SCAMS/ID THEFT         1
Name: Case Type Final Desc, Length: 511, dtype: int64

In [166]:
df['Case Type Final Desc'].astype('category').value_counts()

--DISTURBANCE - OTHER                       559251
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON    459392
--PREMISE CHECKS - CRIME PREVENTION         420320
--ASSIST PUBLIC - OTHER (NON-SPECIFIED)     256256
--TRAFFIC - MOVING VIOLATION                229928
                                             ...  
AWOL - ADULT OR JUVENILE                         1
DV -VULNERABLE ADULT, PHYSICAL ABUSE             1
KNOWN KIDNAPPNG                                  1
TRU - ILLEGAL DUMPING                            1
SECONDARY - FORGERY/BUNCO/SCAMS/ID THEFT         1
Name: Case Type Final Desc, Length: 511, dtype: int64

In [172]:
x=pd.Series(['#NAME?', 'sdfsd', 'dfsf'])
x.str.replace('#NAME\\?', '-')

0        -
1    sdfsd
2     dfsf
dtype: object